In [2]:
import numpy as np
import scipy as sci
import matplotlib.pyplot as plt
from jinja2 import Template
from pyne import data

/home/zoe/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  """
/home/zoe/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: QAWarning: pyne.data is not yet QA compliant.
  return f(*args, **kwds)


First we want the helium density in moles per volume.

\begin{align}
PV&=nRT\\
\frac{n}{V}&=\frac{P}{RT}
\end{align}

In [6]:

R = 8.314 # j/k-mol
temp = [250 ,750] #celsius
pres = [6*10**6, 5.84*10**6] #pascals
hedensity = np.zeros(len(temp))

#serpent needs 10^24/cc

for i, t in enumerate(temp):
    hedensity[i] = ((pres[i])/((t+273.15)*R)) # mol/m^3 

hedensavg = np.sum(hedensity)/len(hedensity)*(6.022*10**-7)# 10^24/cc

print(hedensavg)

0.000622077124202638


In [5]:
#calculate isotopic comp. of uranium isotopes for UCO
#data on properties of UCO from Nagley et. all "Fabrication..."
#update this using pyne

molUO2 = 0.714
molUC86 = 0.123
molUC100 = 0.164

mu235 = 235.043
mu238 = 238.050
mo16 = 15.994
mc12 = 12.000

uenrich = 0.15

massUO2 = (uenrich*mu235) + (1-uenrich)*mu238 + (2*mo16)
massUC86 = (uenrich*mu235) + (1-uenrich)*mu238 + .86*mc12 #actually, should this be 1.86, or .86?
massUC100 = (uenrich*mu235) + (1-uenrich)*mu238 + mc12

massUCO = molUO2*massUO2 + molUC86*massUC86 + molUC100*massUC100

o16wt = (2*molUO2*mo16)/(massUCO)
u235wt = (uenrich*mu235)/(massUCO)
u238wt = ((1-uenrich)*mu238)/(massUCO)
c12wt = (molUC86*.86*mc12 + molUC100*mc12)/(massUCO)

print(u235wt, u238wt, o16wt, c12wt)

0.13359101087155556 0.7667005361367276 0.08654140756123074 0.012266753883477748


In [2]:
molUO2 + molUC86 + molUC100


1.001

In order to find the core dimensions, I took an image of the reactor core given by X-energy in a presentation to the NRC.  
<br>
Then, I opened it in an image viewer that gave the cursor's coordinates in pixels, measured the reactor core features in pixels.  Knowing the outer core radius, developed a scale between pixels and meters, and used that to determine the core geometry.

In [11]:
core_dm = 4.88 # m
core_dpx = 69 #px
scale = core_dm/core_dpx *100 # cm/px

inner_corerad_px = 17.5
inner_corerad_cm = inner_corerad_px *scale #convert to cm
reflect_px = 13
reflect_cm = reflect_px *scale

print(2*(reflect_cm+inner_corerad_cm)) #can't be more than 488, which is the diameter including RPV
print()
print(inner_corerad_cm)
print(reflect_cm)
print(inner_corerad_cm+reflect_cm)

431.4202898550724

123.76811594202897
91.94202898550724
215.7101449275362


I found the midpoint of the cones on the top and bottom , and used the distance between these two midpoints as an adjusted height.  Then, I simplified by letting the reflector directly surround this region.

In [10]:
mid_bott_y = 228
mid_top_y = 91.5

adj_h_px = mid_bott_y - mid_top_y #px

adj_h_cm = adj_h_px*scale

print(adj_h_cm)

965.391304347826
